In [1]:
import os
import requests
import datetime

from utils.contract_utils import init_contract
from utils.network_utils import configure_network_and_connect

import brownie

import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
sns.set_style("whitegrid")

ALCHEMY_API_KEY = os.environ['ALCHEMY_API_KEY']
configure_network_and_connect(
    node_provider_https=f"https://eth-mainnet.alchemyapi.io/v2/{ALCHEMY_API_KEY}",
    network_name='mainnet'
)

In [2]:
pool_address = "0x752eBeb79963cf0732E9c0fec72a49FD1DEfAEAC"

In [3]:
api = "https://api.thegraph.com/subgraphs/name/convex-community/curve-factory-volume"
query = """
{
  swapEvents(
    first: 1000,
    orderBy: timestamp,
    orderDirection: desc,
    where: {
      pool: "0x752ebeb79963cf0732e9c0fec72a49fd1defaeac"
    }
  )
  {
    block
    timestamp
  }
}
"""
r = requests.post(api, json={'query': query})
data = dict(r.json())
blocks = [int(i['block']) for i in data['data']['swapEvents']]
timestamps = [int(i['timestamp']) for i in data['data']['swapEvents']]
timestamps = [datetime.datetime.fromtimestamp(i) for i in timestamps]

In [4]:
pool = init_contract(pool_address)
tricrypto2 = init_contract("0xD51a44d3FaE010294C616388b506AcdA1bfAAE46")

In [ ]:
price_oracle_data = []
for block in blocks:

    price_oracle = pool.price_oracle(block_identifier=block) * 1e-18
    eth_price = tricrypto2.price_oracle(1, block_identifier=block) * 1e-18
    price_oracle_data.append(price_oracle * eth_price)

price_oracle_data

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(30, 10))

ax.plot(
    timestamps,
    price_oracle_data,
    'kx-',
    lw=3,
    ms=10,
    label='Threshold Network (T)'
)

ax.set_ylabel("Curve v2 Token Price [in USD]", fontsize=25)
ax.set_xlabel("Block Number", fontsize=25)
ax.tick_params(axis='both', which='major', labelsize=25)
ax.tick_params(axis='both', which='major', labelsize=25, colors='black')
ax.legend(fontsize=25);